In [1]:
!nvidia-smi

Sun May 12 03:16:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      On  | 00000000:00:03.0 Off |                    0 |
| N/A   67C    P8              19W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/home/jupyter/unattended-baggage-yolov8-modification


In [3]:
!pip install opencv-python
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.5 MB/s eta 0:00:00


In [1]:
!pip install -q \
clearml

In [13]:
!pip install -e .

Obtaining file:///home/jupyter/unattended-baggage-yolov8-modification
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.2-0.editable-py3-none-any.whl size=16601 sha256=01a714cb67710d0bd4f7bed197a18bfae3defe2430d545d512f614a3413993b3
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-0uxqm91l/wheels/16/24/4e/77eb3faafdc0df8f4050b5c10cf91dc4af4801edeb0d66c251
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.2
    Uninstalling ultralytics-8.2.2:
      Successfully uninstalled ultralytics-8.2.2


## Setup ClearML

In [2]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
%env CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
env: CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4


## YOLOv8 baseline

In [11]:
task.close()

In [12]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="baseline 5 May 2024 Model Training YOLOv8",
)

ClearML Task: created new task id=889f4ac60a10442886625cb324e3a3d8
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/889f4ac60a10442886625cb324e3a3d8/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [13]:
# Step 2: Selecting the YOLOv8 Model, for this case baseline, yolov8n
model_variant = "yolov8n"
task.set_parameter("model_variant", model_variant)

In [14]:
# # Step 3: Loading the YOLOv8 Model
from ultralytics import YOLO
model = YOLO(f'{model_variant}.pt')

Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


## YOLOv8 configuration used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size 
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for baseline training

In [15]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'baseline-unattended_baggage_detection_yolov8',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'baseline-unattended_baggage_detection_yolov8',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [16]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline-unattended_baggage_detection_yolov8/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline-unattended_baggage_detection_yolov8
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100     0.795G     0.7028     0.4318     0.9205          2       1280: 100%|██████████| 145/145 [00:29<00:00,  4.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.44it/s]


                   all        145        586       0.89      0.765      0.848      0.584

100 epochs completed in 0.947 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline-unattended_baggage_detection_yolov8/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline-unattended_baggage_detection_yolov8/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline-unattended_baggage_detection_yolov8/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUD

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:03<00:00, 18.46it/s]


                   all        145        586      0.864      0.775      0.851      0.586
              backpack        145        192      0.837      0.804      0.832      0.522
               handbag        145        310      0.867      0.684      0.819      0.467
              suitcase        145         84      0.886      0.836      0.902      0.769
Speed: 0.4ms preprocess, 23.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline-unattended_baggage_detection_yolov8
2024-05-05 03:28:28,930 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/baseline%205%20May%202024%20Model%20Training%20YOLOv8.889f4ac60a10442886625cb324e3a3d8/models/best.pt


Connecting multiple input models with the same name: `best`. This might result in the wrong model being used when executing remotely


## YOLOv8 +AIFI backbone

In [13]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="AIFI 12 May 2024 Model Training",
)


ClearML Task: created new task id=676510d0a2eb4b6e865e20fc23c1fbd0
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/676510d0a2eb4b6e865e20fc23c1fbd0/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [16]:
# Step 2: Selecting the YOLOv8 Model, for this case AIFI
model_variant = "ultralytics/cfg/models/v8/yolov8-aifi.yaml"
task.set_parameter("model_variant", model_variant)

In [17]:
# Step 3: Loading the YOLOv8 AIFI Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


Transferred 66/515 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

## YOLOv8 AIFI Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_aifi #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [18]:

# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=-1,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 8,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'AIFI_unattended_baggage_detection',
                      save = True,
                      plots = True,
                      patience = 30,
                      verbose = True)
task.connect(args)


{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': -1,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 8,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'AIFI_unattended_baggage_detection',
 'save': True,
 'plots': True,
 'patience': 30,
 'verbose': True}

In [19]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.13 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-aifi.yaml, data=/home/jupyter/gcs/unattended-

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.25.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_unattended_baggage_detection/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 83 weight(decay=0.0), 94 weight(decay=0.0005), 95 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 64 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_unattended_baggage_detection
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      3.83G     0.7149     0.4819      1.203          2       1280: 100%|██████████| 73/73 [00:25<00:00,  2.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:03<00:00,  9.34it/s]


                   all        145        586      0.878      0.787       0.86      0.579

100 epochs completed in 0.887 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_unattended_baggage_detection/weights/last.pt, 33.0MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_unattended_baggage_detection/weights/best.pt, 33.0MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_unattended_baggage_detection/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
     

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:03<00:00,  9.51it/s]


                   all        145        586      0.925      0.762      0.871      0.587
              backpack        145        192      0.907      0.771      0.861      0.536
               handbag        145        310       0.92      0.707      0.873      0.499
              suitcase        145         84      0.947       0.81      0.879      0.726
Speed: 0.5ms preprocess, 22.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_unattended_baggage_detection
2024-05-12 07:39:07,100 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/AIFI%2012%20May%202024%20Model%20Training.676510d0a2eb4b6e865e20fc23c1fbd0/models/best.pt


Connecting multiple input models with the same name: `best`. This might result in the wrong model being used when executing remotely


## YOLOv8 + BilevelRoutingAttention

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="3 May 2024 Model Training BiLevelRouting Attention",
)


ClearML Task: created new task id=616edbb8c0484e29a606b2d2c38ea22a
2024-05-03 08:11:28,572 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/616edbb8c0484e29a606b2d2c38ea22a/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case BilevelRoutingAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 BilevelRoutingAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-03 08:12:03,967 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 186/379 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 BiLevelRouting Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 8  # Smaller batch size for Tesla T4
device = 0,1,2,3,4,5,6,7 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_bilevelroutingattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_bilevelroutingattention_multihead',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_bilevelroutingattention_multihead',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml, data=/home/jupyt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.789      0.668      0.708      0.446
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     91/100      8.49G       1.05     0.8027       1.23          1       1280: 100%|██████████| 145/145 [00:36<00:00,  3.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.41it/s]


                   all        145        586      0.744      0.662      0.681      0.406
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 66, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

91 epochs completed in 1.042 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/last.pt, 7.2MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/best.pt, 7.2MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 16.85it/s]


                   all        145        586      0.855      0.698      0.802      0.495
              backpack        145        192      0.802      0.719       0.79      0.457
               handbag        145        310      0.866      0.635       0.78      0.388
              suitcase        145         84      0.897      0.738      0.836      0.641
Speed: 0.4ms preprocess, 25.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead
2024-05-03 09:17:18,404 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/3%20May%202024%20Model%20Training%20BiLevelRouting%20Attention.616edbb8c0484e29a606b2d2c38ea22a/models/best.pt


### YOLOv8 + Shuffle Attention

In [11]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="Shuffle Attention 3 May 2024 Model Training Multi Head",
)


ClearML Task: created new task id=ef6340197a6e45ceaf83fe6ffab81685
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/ef6340197a6e45ceaf83fe6ffab81685/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [12]:
# Step 2: Selecting the YOLOv8 Model, for this case ShuffleAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [13]:
# Step 3: Loading the YOLOv8 ShuffleAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


Transferred 186/379 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 Shuffle Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_shuffleattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [14]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_shuffleattention_multihead',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_shuffleattention_multihead',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [15]:
# Step 5: Initiating Model Training for ShuffleAttention
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml, data=/home/jupyter/gcs/

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 61 weight(decay=0.0), 72 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     94/100     0.799G     0.9945     0.7234      1.182          1       1280: 100%|██████████| 145/145 [00:32<00:00,  4.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.02it/s]


                   all        145        586       0.84      0.672      0.777      0.485
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 69, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

94 epochs completed in 0.968 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                    

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 18.11it/s]


                   all        145        586      0.816      0.716      0.786      0.487
              backpack        145        192      0.856      0.713      0.769      0.437
               handbag        145        310      0.708        0.7      0.776      0.399
              suitcase        145         84      0.885      0.735      0.814      0.626
Speed: 0.4ms preprocess, 23.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead
2024-05-03 10:23:24,471 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/Shuffle%20Attention%203%20May%202024%20Model%20Training%20Multi%20Head.ef6340197a6e45ceaf83fe6ffab81685/models/best.pt


Connecting multiple input models with the same name: `best`. This might result in the wrong model being used when executing remotely


## YOLOv8 + SPD-Conv

In [3]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="SPD-Conv 12 May 2024 Model Training",
)


ClearML Task: created new task id=b47fd6c49bac4d8db43d86eef57468e1
2024-05-12 03:21:48,544 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/b47fd6c49bac4d8db43d86eef57468e1/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [8]:
# Step 2: Selecting the YOLOv8 Model, for this case SPD-Conv
model_variant = "ultralytics/cfg/models/v8/yolov8-SPD-Conv.yaml"
task.set_parameter("model_variant", model_variant)

In [9]:
# Step 3: Loading the YOLOv8 SPD-Conv Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-12 03:26:32,241 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 52/445 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): space_to_depth()
      (3): C2f(
        (cv1): Conv(
          (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03

In [10]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=-1,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 8,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_SPD_Conv',
                      save = True,
                      plots = True,
                      patience = 30,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': -1,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 8,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_SPD_Conv',
 'save': True,
 'plots': True,
 'patience': 30,
 'verbose': True}

In [11]:
# Step 5: Initiating Model Training for SPD-Conv
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.13 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-SPD-Conv.yaml, data=/home/jupyter/gcs/unatten

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.27.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 72 weight(decay=0.0), 79 weight(decay=0.0005), 78 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 64 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.925      0.755      0.868      0.597

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      16.1G     0.6663     0.4292      1.139          2       1280: 100%|██████████| 73/73 [01:23<00:00,  1.15s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:16<00:00,  2.26it/s]


                   all        145        586      0.905       0.77      0.873      0.601

100 epochs completed in 2.834 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv/weights/last.pt, 24.1MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv/weights/best.pt, 24.1MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 224

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:16<00:00,  2.31it/s]


                   all        145        586      0.904      0.769      0.879      0.601
              backpack        145        192      0.927      0.812      0.881      0.555
               handbag        145        310      0.958      0.734      0.891      0.526
              suitcase        145         84      0.827      0.762      0.867      0.723
Speed: 0.5ms preprocess, 105.6ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv
2024-05-12 06:22:04,486 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/SPD-Conv%2012%20May%202024%20Model%20Training.b47fd6c49bac4d8db43d86eef57468e1/models/best.pt


## YOLOv8 + SPD-Conv + AIFI

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="SPD-Conv+AIFI 5 May 2024 Model Training",
)


ClearML Task: created new task id=80bebd3ebbd5460eb0b7081ceb7964d9
2024-05-05 00:23:51,292 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/80bebd3ebbd5460eb0b7081ceb7964d9/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case AIFI + SPD-Conv
model_variant = "ultralytics/cfg/models/v8/yolov8-aifi-SPD-Conv.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 AIFI + SPD-Conv Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-05 00:24:05,806 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 29/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): space_to_depth()
      (3): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03,

In [7]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_AIFI_SPD-Conv',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_AIFI_SPD-Conv',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [8]:
# Step 5: Initiating Model Training for AIFI + SPD-Conv
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-aifi-SPD-Conv.yaml, data=/home/jupyter/gcs/una

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.28.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 66 weight(decay=0.0005), 67 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.881      0.688      0.799      0.535

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      7.65G     0.8479     0.6028      1.219          2       1280: 100%|██████████| 145/145 [00:54<00:00,  2.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:05<00:00, 12.53it/s]


                   all        145        586      0.888      0.687      0.804      0.541

100 epochs completed in 1.687 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/weights/last.pt, 8.8MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/weights/best.pt, 8.8MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:05<00:00, 12.99it/s]


                   all        145        586        0.9       0.73       0.83      0.552
              backpack        145        192      0.837      0.724      0.808      0.498
               handbag        145        310      0.896      0.732      0.834      0.478
              suitcase        145         84      0.969      0.733      0.849       0.68
Speed: 0.4ms preprocess, 34.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD-Conv2
2024-05-05 02:07:02,223 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/SPD-Conv%2BAIFI%205%20May%202024%20Model%20Training.80bebd3ebbd5460eb0b7081ceb7964d9/models/best.pt
